# Пример исследования эффективности рекламы приложения
* Расчёт ARPPU;
* Расчёт ROAS;
* Прогноз окупаемости рекламной кампании;
* Построение маркетинговой воронки;
* Сегментация пользователей.


### Описание данных:

Файл db.db — это файл базы данных sqlite. В базе собрана статистика по когортам пользователей мобильного приложения, привлеченных через рекламные каналы. Каждая когорта определяется тремя признаками **date, country, os**.
В базе есть две таблички **spent** и **payments**.

In [1]:
# Импортируемые модули

import sqlite3
import os 

import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
%%capture
%load_ext sql
%sql sqlite:///E://Practice//db.db

In [3]:
%%sql

select * 
from spent
limit 5;

 * sqlite:///E://Practice//db.db
Done.


date,os,country,spend,impressions,clicks,installs
2020-06-01,android,AD,0.02,32,2,2
2020-06-01,android,AE,2.73,1167,19,8
2020-06-01,android,AF,0.07,110,4,0
2020-06-01,android,AG,0.04,26,0,0
2020-06-01,android,AL,0.13,70,1,1


В табличке **spent** собраны данные о рекламных расходах, просмотрах, кликах и установках. Описание полей из таблицы **spent**:

* date - дата, когда был потрачен бюджет (spend);
* os - операционная система устройства;
* country - страна просмотра рекламы или установки приложения;
* spend - деньги, потраченные на рекламу;
* impressions - количество просмотров рекламного объявления;
* clicks - количество кликов на рекламу;
* installs - количество установок приложения (размер когорты).

In [4]:
%%sql

select * 
from payments
limit 5;

 * sqlite:///E://Practice//db.db
Done.


date,ts,os,country,purchases,unique_purchases,app_revenue
2020-08-04,2020-08-14 04:04:06.608152,ios,US,3,2,9.97
2020-07-28,2020-07-29 04:03:56.516653,ios,NL,0,0,0.0
2020-06-14,2020-07-09 04:02:08.915834,ios,SA,8,2,33.8031
2020-06-10,2020-06-13 04:05:36.297502,android,IQ,0,0,0.0
2020-06-14,2020-06-28 04:04:59.576784,android,LB,0,0,0.0


В табличке **payments** собраны данные о действиях пользователей каждой когорты после установки (с 1 по 28 день после установки). Данные собираются из трекинговой системы и записываются в таблицу в начале каждого дня. До тех пор, пока ts - date < 29 days. Описание полей из таблицы **payments**:

* date - дата установки приложения
* ts - Момент сбора данных из трекинговой системы. Пример: строку со значениями [2020-06-11 2020-07-22 08:05:22.070317 ios FR 8 3 33.77820] можно интерпретировать как "в период с 2020-06-11 по 2020-07-22 08:05:22.070317, 3 пользователя из когорты [2020-06-11, FR, ios] совершили 8 покупок на общую сумму 33.78".
* os - операционная система устройства;
* country - страна установки;
* purchases - количество совершенных покупок в приложении;
* unique_purchases - количество уникальных пользователей, совершивших покупку.
* app_revenue - деньги, полученные приложением за покупку.

## ARPPU

ARPPU (AVERAGE REVENUE PER PAYING USER) - отношение полученных денег к количеству уникальных покупок (пользователей) из таблицы payments, где ts - максимально.

In [5]:
%%sql

select os, sum(app_revenue) / sum(unique_purchases) as ARPPU  
from payments
    where ts in 
    (select max(ts) from payments
     group by date, os, country)
group by os;

 * sqlite:///E://Practice//db.db
Done.


os,ARPPU
android,16.97853792445534
ios,26.295473604467205


## ROAS
ROAS (RETURN ON ADVERTISEMENT SPENT) - отношение дохода от рекламной кампании к расходам на неё.
Так как в наших данных доходы регистрируются вплоть до 28 дня включительно, а доход считается накопительным итогом, то актуальным показателем для расчёта ROAS будет доход в последний день.

В первую очередь отберём данные, где разница между ts и date равняется 28 дням. Также проверим, какой тип join делать - есть ли нулевые записи spend при ненулевых доходах и, если такие есть - остаётся вопрос, учитывать ли их в финальном запросе.

In [6]:
%%sql

select p.date, p.ts, p.os, p.country, p.app_revenue, s.spend
from payments p

left join spent s on s.os = p.os and s.date = p.date and s.country = p.country
where cast(JulianDay(p.ts) - JulianDay(p.date) as integer) = 28
    and s.spend is null and app_revenue > 0
    
limit 5;

 * sqlite:///E://Practice//db.db
Done.


date,ts,os,country,app_revenue,spend
2020-06-15,2020-07-13 04:04:05.205915,ios,SA,76.8349,None
2020-07-13,2020-08-10 04:04:06.726223,android,TR,22.6888,None
2020-06-15,2020-07-13 04:04:05.205915,android,GR,12.3555,None
2020-06-22,2020-07-20 04:04:06.116495,android,IE,6.7668,None
2020-06-15,2020-07-13 04:04:05.205915,ios,BR,14.4016,None


Делаем итоговый запрос, который возваращает данные из таблицы payments с левым присоединением таблицы spent. Нас интересует os, месяц и отношение полученных денег к затратам на рекламную кампанию с группировкой по os и месяцам.

In [7]:
%%sql

select  p.os as os, 
        strftime('%m', p.date) as month, 
        sum(app_revenue) / sum(coalesce(spend, 0)) as roas_28d
from payments p
left join spent s on s.os = p.os and s.date = p.date and s.country = p.country
where cast(JulianDay(p.ts) - JulianDay(p.date) as integer) = 28
group by p.os, month;

 * sqlite:///E://Practice//db.db
Done.


os,month,roas_28d
android,06,0.8911538182041718
android,07,0.6595421298710297
ios,06,0.8497541894836711
ios,07,1.187854000350659


Можно было усложнить запрос и показать темп роста силами sql, однако, учитывая, что необходима запись в файл - работа с python, я сделал это вычисление в питоне.

## Прогноз ROAS
У нас есть данные roas_28d с 1го июня по 18 июля. После 18 июля есть данные по spend и нет данных по app_revenue за полные 28 дней кампании, их и надо спрогнозировать. 
Можно выгрузить данные за имеющийся срок отдельно с группировкой по дням, построить регрессионную модель и проверить насколько день месяца и параметр spend влияют на доход. Если гипотеза, что эти параметры независимы не подтвердится, то по данным параметра spend за вторую половину июля можно будет спрогнозировать динамику параметра app_revenue.

Выгрузим данные, для которых мы можем подсчитать roas_28d с группировкой по дням и отдельно данные затрат после 18 июля.

In [8]:
# Данные с построения модели

db = os.path.join('E:','Practice','db.db')
conn = sqlite3.connect(db)
cursor = conn.cursor()

data = cursor.execute('''
select  p.os as os, 
        strftime('%m', p.date) as month,
        strftime('%d', p.date) as day,
        sum(app_revenue),
        sum(coalesce(spend, 0)) tt
        
from payments p
left join spent s on s.os = p.os and s.date = p.date and s.country = p.country
where cast(JulianDay(p.ts) - JulianDay(p.date) as integer) = 28
group by p.os, month, day''').fetchall()

# Данные с расходами после 18.07
test = cursor.execute('''
select os,
       strftime('%m', date) as month,
       strftime('%d', date) as day,
       sum(coalesce(spend, 0))
from spent
where day > "18" and month = "07"
group by os, month, day

''').fetchall()

In [9]:
# Загрузка данных в датафреймы, переименование столбцов, исправление типов данных

df_main = pd.DataFrame(data)
df_main.columns = ['os', 'month', 'day', 'app_revenue', 'spend']

df_test = pd.DataFrame(test)
df_test.columns = ['os', 'month', 'day', 'spend']

df_main['month'] = df_main['month'].astype(int)
df_main['day'] = df_main['day'].astype(int)
df_main['app_revenue'] = df_main['app_revenue'].astype(float)
df_main['spend'] = df_main['spend'].astype(float)

df_test['month'] = df_test['month'].astype(int)
df_test['day'] = df_test['day'].astype(int)

In [10]:
# Разделим датасеты по платформам
df_android = df_main[df_main['os'] == 'android']
df_ios = df_main[df_main['os'] == 'ios']
df_test_android = df_test[df_test['os'] == 'android']
df_test_ios = df_test[df_test['os'] == 'ios'] 

In [11]:
# Сделаем модель для андроида
model_android = ols('app_revenue ~ day + spend', data=df_android).fit()
print(model_android.summary())

                            OLS Regression Results                            
Dep. Variable:            app_revenue   R-squared:                       0.945
Model:                            OLS   Adj. R-squared:                  0.942
Method:                 Least Squares   F-statistic:                     318.9
Date:                Wed, 11 Nov 2020   Prob (F-statistic):           4.71e-24
Time:                        14:41:07   Log-Likelihood:                -267.88
No. Observations:                  40   AIC:                             541.8
Df Residuals:                      37   BIC:                             546.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -39.2460    120.042     -0.327      0.7

Согласно модели день месяца не влияет на полученный доход от кампании (по показателям t, P), тогда как параметр spend - напрямую влияет на доход. Предупреждение о высокой степени мультиколлениарности касается показателя spend и указывает на прямую зависимость между ним и зависимой переменной. Таким образом мы можем спрогнозировать app_revenue второй половины июля с помощью имеющейся модели и данных по затратам для этого периода.

In [12]:
# Перестроим модель без переменной 'day'
model_android = ols('app_revenue ~ spend', data=df_android).fit()
print(model_android.summary())

                            OLS Regression Results                            
Dep. Variable:            app_revenue   R-squared:                       0.945
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                     654.9
Date:                Wed, 11 Nov 2020   Prob (F-statistic):           1.46e-25
Time:                        14:41:07   Log-Likelihood:                -267.88
No. Observations:                  40   AIC:                             539.8
Df Residuals:                      38   BIC:                             543.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -40.6626     40.582     -1.002      0.3

In [13]:
# Аналогично сделаем модель для ios
model_ios = ols('app_revenue ~ spend', data=df_ios).fit()
print(model_ios.summary())

                            OLS Regression Results                            
Dep. Variable:            app_revenue   R-squared:                       0.789
Model:                            OLS   Adj. R-squared:                  0.783
Method:                 Least Squares   F-statistic:                     130.7
Date:                Wed, 11 Nov 2020   Prob (F-statistic):           2.29e-13
Time:                        14:41:07   Log-Likelihood:                -275.96
No. Observations:                  37   AIC:                             555.9
Df Residuals:                      35   BIC:                             559.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     98.7146     96.955      1.018      0.3

In [14]:
# Прогноз app_revenue по столбцу затрат
prediction_android = model_android.get_prediction(df_test_android['spend']).summary_frame()
prediction_ios = model_ios.get_prediction(df_test_ios['spend']).summary_frame()

In [15]:
# Итоговый roas_28d за июль по платформам, рассчитанный как отношение суммы прогноза и реального дохода в июле к аналогичным расходам
roas_28d_android_july = (prediction_android['mean'].sum() + df_android[df_android['month'] == 7]['app_revenue'].sum()) / (df_test_android['spend'].sum() + df_android[df_android['month'] == 7]['spend'].sum())
roas_28d_ios_july = (prediction_ios['mean'].sum() + df_ios[df_ios['month'] == 7]['app_revenue'].sum()) / (df_test_ios['spend'].sum() + df_ios[df_ios['month'] == 7]['spend'].sum())

print(f'Прогнозируемое значение roas_28d для андроид за июль: {roas_28d_android_july:.4f}')
print(f'Прогнозируемое значение roas_28d для ios за июль: {roas_28d_ios_july:.4f}')

Прогнозируемое значение roas_28d для андроид за июль: 0.7104
Прогнозируемое значение roas_28d для ios за июль: 1.1857


## Построение воронки и сегментация пользователей
Допустим, нам нужно дать рекомендации отделу по закупке трафика. Для этого ответим на ряд вопросов:
* какая динамика наблюдается в целом? (рекомендуем построить воронку конверсий от impression до конверсии в покупку)
* какие сегменты имеют хорошие показатели и могут масштабироваться;
* какие сегменты находятся в серой зоне и что можно сделать для улучшения их перфоманса;
* какие сегменты скорее всего безнадежны и на них не стоит тратить бюджеты.

` Допущение: норма roas_28d = 1.2`

In [16]:
# Выгружаем данные из базы с преобразованием в pandas dataframe. Используем только данные с полным периодом кампании (28 дней)
data = pd.DataFrame(cursor.execute('''
select  p.os as os,
        p.country,
        p.date,
        strftime('%m', p.date) as month,
        strftime('%d', p.date) as day,
        app_revenue,
        coalesce(spend, 0),
        impressions,
        clicks,
        installs,
        purchases,
        unique_purchases
        
from payments p
left join spent s on s.os = p.os and s.date = p.date and s.country = p.country
where cast(JulianDay(p.ts) - JulianDay(p.date) as integer) = 28''').fetchall())

# Переименовываем столбцы 
data.columns = ['os', 
                'country', 
                'date', 
                'month', 
                'day', 
                'revenue', 
                'spend', 
                'impressions', 
                'clicks', 
                'installs', 
                'purchases', 
                'unique_purchases']

In [17]:
# Построим воронку по полному набору данных с разделением на 3 показателя: натуральные значения, % от impressions и % от предыдущего этапа воронки

print(f"stage {'abs':>17} / {'% of impressions':>0} / {'% of previous':>5}")
print(f"-" * 58)
print(f"Impressions: {data['impressions'].sum():>10.0f} / {data['impressions'].sum() / data['impressions'].sum():>16.2%} /")
print(f"Clicks ~ CTR: {data['clicks'].sum():>9.0f} / {data['clicks'].sum() / data['impressions'].sum():>16.2%} /")
print(f"Installs ~ CR: {data['installs'].sum():>8.0f} / {data['installs'].sum() / data['impressions'].sum():>16.2%} / {data['installs'].sum() / data['clicks'].sum():>12.2%}")
print(f"Paying users: {data['unique_purchases'].sum():>9.0f} / {data['unique_purchases'].sum() / data['impressions'].sum():>16.2%} / {data['unique_purchases'].sum() / data['installs'].sum():>12.2%}")
print(f"Purchases: {data['purchases'].sum():>12.0f} / {data['purchases'].sum() / data['impressions'].sum():>16.2%} / {data['purchases'].sum() / data['unique_purchases'].sum():>12.2%}")

stage               abs / % of impressions / % of previous
----------------------------------------------------------
Impressions:   14561052 /          100.00% /
Clicks ~ CTR:    251172 /            1.72% /
Installs ~ CR:    84876 /            0.58% /       33.79%
Paying users:      2480 /            0.02% /        2.92%
Purchases:        15462 /            0.11% /      623.47%


В среднем 1,72% человек переходят по рекламной ссылке, из них 33,79% (0,58% от видевших рекламу) устанавливают приложение. Покупки внутри приложения совершают 2,92% от количества людей, установивших приложение или 0,02% от увидевших рекламу. В среднем на одного активного "покупателя" приходится 6,235 покупок, таким образом количество продаж в приложении составляет 0,11% от показов рекламы.

In [18]:
# Для сегментации перегруппируем датафрейм, выделив в индекс когорты: платформа, страна и дата
segments = data.groupby(['country', 'os', 'date']).sum()

# Подозрительные данные, в которых отсутствует информация из таблицы 'spent'
mask = (segments['revenue'] > 0) & (segments['spend'] == 0)
segments[mask].head()

revenue  spend  impressions  clicks  installs  \
country os      date                                                         
BR      android 2020-06-20   3.94422    0.0          0.0     0.0       0.0   
        ios     2020-06-15  14.40160    0.0          0.0     0.0       0.0   
CA      ios     2020-06-16  40.68060    0.0          0.0     0.0       0.0   
DE      ios     2020-06-15   1.22404    0.0          0.0     0.0       0.0   
GR      android 2020-06-15  12.35550    0.0          0.0     0.0       0.0   

                            purchases  unique_purchases  
country os      date                                     
BR      android 2020-06-20          3                 1  
        ios     2020-06-15          6                 1  
CA      ios     2020-06-16         14                 1  
DE      ios     2020-06-15          1                 1  
GR      android 2020-06-15          2                 1

In [19]:
# Исключим их и перегруппируем датафрейм для сегментации без дат
segments = segments[~mask]
segments = segments.groupby(['country', 'os']).sum()

# Посчитаем roas_28d для последующей аналитики
segments['roas_28d'] = segments['revenue'] / segments['spend']

# Проверим нулевые значения для образованного столбца
print(segments[segments['roas_28d'].isna()])

                 revenue  spend  impressions  clicks  installs  purchases  \
country os                                                                  
AI      android      0.0    0.0          0.0     0.0       0.0          0   
BT      android      0.0    0.0          0.0     0.0       0.0          0   
CU      android      0.0    0.0          0.0     0.0       0.0          0   
FM      android      0.0    0.0          0.0     0.0       0.0          0   
GM      android      0.0    0.0          0.0     0.0       0.0          0   
HT      android      0.0    0.0          0.0     0.0       0.0          0   
IR      android      0.0    0.0          0.0     0.0       0.0          0   
        ios          0.0    0.0          0.0     0.0       0.0          0   
KN      android      0.0    0.0          0.0     0.0       0.0          0   
MW      android      0.0    0.0          0.0     0.0       0.0          0   
        ios          0.0    0.0          0.0     0.0       0.0          0   

In [20]:
# Проверим какие значения revenue соответствуют нулевым значениям roas_28d
segments[segments['roas_28d'].isna()]['revenue'].value_counts()

0.0    15
Name: revenue, dtype: int64

Так как отличных от нуля значений показателя доходности в нулевых столбцах roas_28d нет, можно оставить всё как есть.

### Сегментация пользователей
1) Сегменты, которые имеют хорошие показатели и могут масштабироваться: roas_28d >= 1.2

In [21]:
# Выделим лучший сегмент в отдельную переменную
top_segment = segments[segments['roas_28d'] >= 1.2]

print('Верх списка лучшего сегмента:\n', top_segment.index.to_list()[:5])
print()
print(f"Среднее значение roas_28d для этого сегмента: {top_segment['revenue'].sum() / top_segment['spend'].sum():.4f}")
print(f"Размер этого сегмента: {top_segment['installs'].sum():.0f} человек")

Верх списка лучшего сегмента:
 [('AE', 'ios'), ('AS', 'android'), ('AU', 'android'), ('AU', 'ios'), ('AZ', 'android')]

Среднее значение roas_28d для этого сегмента: 1.7289
Размер этого сегмента: 8689 человек


2) Наименее перспективный сегмент (не было покупок за время отслеживания результатов)




In [22]:
# Скорее всего безнадежны и на них не стоит тратить бюджеты
low_segment = segments[segments['roas_28d'] == 0]

print('Верх списка худшего сегмента:\n', low_segment.index.to_list()[:5])
print()
print(f"Размер этого сегмента: {low_segment['installs'].sum():.0f} человек")

Верх списка худшего сегмента:
 [('AD', 'android'), ('AF', 'android'), ('AF', 'ios'), ('AG', 'android'), ('AL', 'android')]

Размер этого сегмента: 2588 человек


3) "Серые сегменты"

Для среднего сегмента я предлагаю также использовать CR (конверсию показов в установки приложения), так этот сегмент дополнительно разделится на 2: в одном будут группы, где люди лучше среднего отреагировали на рекламу, в другом, соответственно, на уровне среднего и хуже. Такое разделение позволит сделать дополнительный вывод по самой рекламе, обходя предложения по увеличению ROAS за счёт акций внутри приложения.

In [23]:
# Сделаем булеву маску, с помощью которой будем сравнивать CR сегменты со средними значениями по выборке.
mask = (segments['installs'] / segments['impressions'] > data['installs'].sum() / data['impressions'].sum())

In [24]:
# Среднее значение ROAS среднего сегмента без использования маски CR
segment_x = segments[(segments['roas_28d'] > 0) & (segments['roas_28d'] < 1.2)]
segment_x['revenue'].sum() / segment_x['spend'].sum()

0.7766056869794397

In [25]:
# Построим воронку для нижнего-среднего сегмента
low_mid_segment = segments[(segments['roas_28d'] > 0) & (segments['roas_28d'] < 1.2) & ~mask]
print(f"Размер этого сегмента: {low_mid_segment['installs'].sum():.0f} человека")
print(f"Средний roas_28d внутри сегмента: {low_mid_segment['revenue'].sum() / low_mid_segment['spend'].sum():.4f}")
print()

print(f"stage {'abs':>17} / {'% of impressions':>0} / {'% of previous':>5}")
print(f"-" * 58)
print(f"Impressions: {low_mid_segment['impressions'].sum():>10.0f} / {low_mid_segment['impressions'].sum() / low_mid_segment['impressions'].sum():>16.2%} /")
print(f"Clicks ~ CTR: {low_mid_segment['clicks'].sum():>9.0f} / {low_mid_segment['clicks'].sum() / low_mid_segment['impressions'].sum():>16.2%} /")
print(f"Installs ~ CR: {low_mid_segment['installs'].sum():>8.0f} / {low_mid_segment['installs'].sum() / low_mid_segment['impressions'].sum():>16.2%} / {low_mid_segment['installs'].sum() / low_mid_segment['clicks'].sum():>12.2%}")
print(f"Paying users: {low_mid_segment['unique_purchases'].sum():>9.0f} / {low_mid_segment['unique_purchases'].sum() / low_mid_segment['impressions'].sum():>16.2%} / {low_mid_segment['unique_purchases'].sum() / low_mid_segment['installs'].sum():>12.2%}")
print(f"Purchases: {low_mid_segment['purchases'].sum():>12.0f} / {low_mid_segment['purchases'].sum() / low_mid_segment['impressions'].sum():>16.2%} / {low_mid_segment['purchases'].sum() / low_mid_segment['unique_purchases'].sum():>12.2%}")

Размер этого сегмента: 33494 человека
Средний roas_28d внутри сегмента: 0.8114

stage               abs / % of impressions / % of previous
----------------------------------------------------------
Impressions:    8250770 /          100.00% /
Clicks ~ CTR:    105406 /            1.28% /
Installs ~ CR:    33494 /            0.41% /       31.78%
Paying users:      1622 /            0.02% /        4.84%
Purchases:         9965 /            0.12% /      614.36%


In [26]:
# Как платят пользователи сегмента относительно среднего
print(f"В среднем покупка в приложении приносит: {data['revenue'].sum() / data['purchases'].sum():.2f}, а покупающий пользователь: {data['revenue'].sum() / data['unique_purchases'].sum():.2f} ")
print(f"В этом сегменте покупка в приложении приносит: {low_mid_segment['revenue'].sum() / low_mid_segment['purchases'].sum():.2f}, а покупающий пользователь: {low_mid_segment['revenue'].sum() / low_mid_segment['unique_purchases'].sum():.2f} ")

В среднем покупка в приложении приносит: 3.61, а покупающий пользователь: 22.50 
В этом сегменте покупка в приложении приносит: 3.76, а покупающий пользователь: 23.07 


Люди в этом сегменте хуже восприняли рекламу или само приложение - CR меньше на 40% относительно среднего по выборке, при этом процент установок после переходов по рекламной ссылке также ниже среднего. Но в этом сегменте в 1,6 раза больше пользователей, совершающих покупки внутри приложения, хоть и меньшее их количество. Относительно стоимости покупок - она сопоставима со средней по всей выборке.

Так как реклама отозвалась хуже среднего и в среднем пользователи совершают меньше покупок можно предположить, что для увеличения окупаемости внутри этого сегмента следует протестировать изменённую рекламу. 

Меньшее количество покупок и стоимость на уровне средней по выборке говорит о том, что акции направленные на пакетные предложения, абонементы может привести к увеличению дохода.

In [27]:
# Построим воронку для верхнего-среднего сегмента
top_mid_segment = segments[(segments['roas_28d'] > 0) & (segments['roas_28d'] < 1.2) & mask]
print(f"Размер этого сегмента: {top_mid_segment['installs'].sum():.0f} человек")
print(f"Средний roas_28d внутри сегмента: {top_mid_segment['revenue'].sum() / top_mid_segment['spend'].sum():.4f}")
print()

print(f"stage {'abs':>17} / {'% of impressions':>0} / {'% of previous':>5}")
print(f"-" * 58)
print(f"Impressions: {top_mid_segment['impressions'].sum():>10.0f} / {top_mid_segment['impressions'].sum() / top_mid_segment['impressions'].sum():>16.2%} /")
print(f"Clicks ~ CTR: {top_mid_segment['clicks'].sum():>9.0f} / {top_mid_segment['clicks'].sum() / top_mid_segment['impressions'].sum():>16.2%} /")
print(f"Installs ~ CR: {top_mid_segment['installs'].sum():>8.0f} / {top_mid_segment['installs'].sum() / top_mid_segment['impressions'].sum():>16.2%} / {top_mid_segment['installs'].sum() / top_mid_segment['clicks'].sum():>12.2%}")
print(f"Paying users: {top_mid_segment['unique_purchases'].sum():>9.0f} / {top_mid_segment['unique_purchases'].sum() / top_mid_segment['impressions'].sum():>16.2%} / {top_mid_segment['unique_purchases'].sum() / top_mid_segment['installs'].sum():>12.2%}")
print(f"Purchases: {top_mid_segment['purchases'].sum():>12.0f} / {top_mid_segment['purchases'].sum() / top_mid_segment['impressions'].sum():>16.2%} / {top_mid_segment['purchases'].sum() / top_mid_segment['unique_purchases'].sum():>12.2%}")

Размер этого сегмента: 40105 человек
Средний roas_28d внутри сегмента: 0.5963

stage               abs / % of impressions / % of previous
----------------------------------------------------------
Impressions:    4176054 /          100.00% /
Clicks ~ CTR:    110193 /            2.64% /
Installs ~ CR:    40105 /            0.96% /       36.40%
Paying users:       469 /            0.01% /        1.17%
Purchases:         2045 /            0.05% /      436.03%


In [28]:
# Как платят пользователи сегмента относительно среднего
print(f"В среднем покупка в приложении приносит: {data['revenue'].sum() / data['purchases'].sum():.2f}, а покупающий пользователь: {data['revenue'].sum() / data['unique_purchases'].sum():.2f} ")
print(f"В этом сегменте покупка в приложении приносит: {top_mid_segment['revenue'].sum() / top_mid_segment['purchases'].sum():.2f}, а покупающий пользователь: {top_mid_segment['revenue'].sum() / top_mid_segment['unique_purchases'].sum():.2f} ")

В среднем покупка в приложении приносит: 3.61, а покупающий пользователь: 22.50 
В этом сегменте покупка в приложении приносит: 2.60, а покупающий пользователь: 11.33 


Люди внутри этого сегмента лучше среднего восприняли рекламу, однако они совершают меньше покупок и на меньшие суммы. Для предложений по этому сегменту, к сожалению, необходим более глубокий анализ.

Как итог, отделу по закупке трафика, я бы рекомендовал однозначно закупать трафик в сегменте `top_segment`, в зависимости от необходимого количества использовать пары страна-платформа из `low_mid_segment` и лишь потом `top_mid_segment`.
А также протестировать альтернативную рекламу на сегменте `low_mid_segment`.